In [1]:
import numpy as np
from scipy.optimize import fsolve

In [5]:
# fixed parameter definitions

beta1 = 1
alpha = -2
gamma0 = 1/2
gamma1 = 1/4
beta2_bar = 4
beta3_bar = 4
sigma2 = 1
sigma3 = 1

# markets and goods
T = 600
J = 4

In [6]:
# 3.1

# x_jt, w_jt are absolute value of iid standard normal variables
x = np.absolute(np.random.standard_normal(size=(J,T)))
w = np.absolute(np.random.standard_normal(size=(J,T)))

unobservable_mean = [0,0]
unobservable_cov = [[1,0.25],[0.25,1]]
unobservables = np.random.multivariate_normal(unobservable_mean, unobservable_cov, size=(J,T))
xi = unobservables[:,:,0]
omega = unobservables[:,:,1]

In [4]:
# 3.2a
